# Visualizing fire scars through false color

### Data Source Description:

- #### landsat8
This raster image comprises of a collection of bands from Landsat Collection 2 - Level 2
Source Date:
Date of Access: 

#### CA Perimeters CAL FIRE NIFC FIRIS
The following data source was obtained from the California Natural Resources Agency. The dataset is a shapefile which includes a layer encompassing three feature layers for CAL FIRE's public incident map. The data itself is sourced from the CAL Fire's Intel Remote Sensing Data, FIRIS wildire perimeters, and NIFC's WFIGS 2024 Interagency Fire Perimeters to Date.
Source link: https://gis.data.cnra.ca.gov/datasets/CALFIRE-Forestry::ca-perimeters-cal-fire-nifc-firis-public-view/about
Source Date: 01/30/2024
Date of Access: 11/19/2025

In [ ]:
import numpy as np
import xarray as xr
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import netCDF4


In [7]:
# Read in Landsat 8 data
landsat8 = xr.open_dataset('data/landsat8-2025-02-23-palisades-eaton.nc')

### Data Exploration

In [ ]:

landsat8.dimensions
landsat8.variables
landsat8.attrs
dir(landsat8)